In [1]:
import pandas as pd
import re

# Company Sample
find business registration and VAT number in dataset

### 1. Load Dataset

In [2]:
# Load Company Sample
company_sample = pd.read_csv('company_sample.csv')
total_row = len(company_sample)

# remove empty snippet
company_sample.dropna(
    subset = ['gsearch_snippet1', 'gsearch_snippet2', 'gsearch_snippet3','gsearch_snippet4','gsearch_snippet5'], 
    how='all',
    inplace=True
)
valid_row = len(company_sample)
print('total row:', total_row )
print('valid row:', valid_row)
company_sample.tail(3)

total row: 5417
valid row: 943


,Company,gsearch_snippet1,gsearch_snippet2,gsearch_snippet3,gsearch_snippet4,gsearch_snippet5
996,Coolshop.nl,NaN,NaN,NaN,BTW-ID-Nr. : DK26457602 VAT: NL 824007943B01 O...,BTW-ID-Nr. : DK26457602 VAT: NL 824007943B01 O...
997,BonjourOutdoor.nl,Overige gegevens KvK: 63609835. BTW: NL8553130...,NaN,NaN,De in het aanbod van producten of diensten gen...,Indien u een bestelling wilt plaatsen als inte...
998,HuisEnTuin.Online,huisentuin.online. is onderdeel van: IsMiDo Ha...,NaN,NaN,huisentuin.online. is onderdeel van: IsMiDo Ha...,BTW ID: NL003232521B12 IBAN Nummer: NL88 RABO ...


### 2. Preprocess Dataset

In [3]:
# Merge Snippet
company_sample['FULL_SNIPPET'] = (
    company_sample['gsearch_snippet1'].astype(str) + ' ' +
    company_sample['gsearch_snippet2'].astype(str) + ' ' +
    company_sample['gsearch_snippet3'].astype(str) + ' ' +
    company_sample['gsearch_snippet4'].astype(str) + ' ' +
    company_sample['gsearch_snippet5'].astype(str) 
)

In [4]:
# Make New Dataframe
comp_snippet = company_sample[['Company','FULL_SNIPPET']].copy()
comp_snippet.head(3)

,Company,FULL_SNIPPET
0,De Bijenkorf,Magazijn 'De Bijenkorf' B.V.. Handelend onder ...
1,Zalando.nl,Contact: www.zalando.nl/contact. Bereikbaarhei...
2,Sliponline,KVK KVK nummer: 17044923 (bekijk). Bank Raboba...


In [5]:
#Remove Symbol in FULL SNIPPET
def preprocess(text:str):
    
    # join nummer
    text = text.replace('nummer.','nummer ')
    text = text.replace('Nummer.','Nummer ')
    text = text.replace(' nummer:','-nummer:')
    text = text.replace(' Nummer:','-Nummer:')
    # join nr
    text = text.replace(' nr.:', '-nr:').replace(' nr. ','-nr:').replace(' nr:','-nr:')
    # remove .
    # text = text.replace('.', '')
    text = re.sub('(\w)\.(\w)', '', text)
    
    
    # remove multiple space
    text = re.sub(' +', ' ', text)
    return text

comp_snippet['CLEAN_SNIPPET'] = comp_snippet['FULL_SNIPPET'].apply(preprocess)

from comp_snippet we will create two branch, comp_snippet_br for business register and comp_snippet_vat for vat.

They will be merged into a single dataframe in the end

# Business Register

### 1. Load Base Knowledge

In [6]:
knowledge = pd.read_csv('KNOWLEDGE.csv')
knowledge.head(2)

,country,business_register,business_register_abb,vat,vat_abb
0,Austria,Firmenbuchnummer,FN,Mehrwertsteuer,MwSt
1,Austria,Unternehmens-Identifikationsnummer,UID,Mehrwertsteuer,MwSt


### 2. Find Business Register

In [7]:
def find_business_register(text):
    registers = []
    for key,row in knowledge.iterrows():
        if row['business_register_abb'].lower() in text.lower() or row['business_register'].lower() in text.lower():
            registers.append(row['business_register_abb'])
    return registers if registers else None

comp_snippet['BUSINESS_REGISTER'] = comp_snippet['CLEAN_SNIPPET'].apply(find_business_register)

In [8]:
comp_snippet_br = comp_snippet.explode(['BUSINESS_REGISTER'])
comp_snippet_br.head(2)

,Company,FULL_SNIPPET,CLEAN_SNIPPET,BUSINESS_REGISTER
0,De Bijenkorf,Magazijn 'De Bijenkorf' B.V.. Handelend onder ...,Magazijn 'De Bijenkorf' .. Handelend onder de ...,KvK
1,Zalando.nl,Contact: www.zalando.nl/contact. Bereikbaarhei...,Contact: wwalandl/contact. Bereikbaarheid: Maa...,HRB


### 3. Find Business Register Country

In [9]:
comp_snippet_br = comp_snippet_br.merge(knowledge[['business_register_abb','business_register','country']],left_on='BUSINESS_REGISTER', right_on='business_register_abb', how='left')

### 4. Find Business Register Number

In [10]:
def find_br_number(row):
    results = []
    breg_abb = row['BUSINESS_REGISTER']
    breg = row['business_register']

    # loop trough business register abbrivation and long version
    bregs = [breg_abb, f'{breg_abb}-nummer', f'{breg_abb}-nr',breg, f'{breg}-nummer', f'{breg}-nr']

    for br in bregs:
        pattern = rf'(?i){br}\s*:?\s*([\w\s]+)'
        results.extend(
            re.findall(pattern,row['CLEAN_SNIPPET'])
        )

    clean_results = []
    for res in results:
        # remove word
        pattern = r'\b(?:[a-zA-Z]{4,})\b'
        res = re.sub(pattern, '', res)

        # remove lowercase word
        pattern = r'\b(?:[a-z]+)\b'
        res = re.sub(pattern, '', res)

        # remove nan
        res = res.replace('nan','').strip()
        
        if len(res) > 2:
            clean_results.append(res)
    return clean_results

comp_snippet_br['BUSINESS_REGISTER_NUM'] = comp_snippet_br.apply(find_br_number,axis=1)
comp_snippet_br.sample(2)

,Company,FULL_SNIPPET,CLEAN_SNIPPET,BUSINESS_REGISTER,business_register_abb,business_register,country,BUSINESS_REGISTER_NUM
916,123led.nl,Bank en KVK gegevens. IBAN: NL21 RABO 0304 513...,Bank en KVK gegevens. IBAN: NL21 RABO 0304 513...,KvK,KvK,Kamer van Koophandel,Netherlands,[]
866,Hornbach.nl,De pas is alleen bedoeld voor bedrijven die in...,De pas is alleen bedoeld voor bedrijven die in...,KvK,KvK,Kamer van Koophandel,Netherlands,[30151645]


In [11]:
comp_snippet_br_expl = comp_snippet_br.explode(['BUSINESS_REGISTER_NUM'])
print(len(comp_snippet_br_expl))
comp_snippet_br_expl.sample(2)

1592


,Company,FULL_SNIPPET,CLEAN_SNIPPET,BUSINESS_REGISTER,business_register_abb,business_register,country,BUSINESS_REGISTER_NUM
256,Baby-Dump,Van maandag t/m zaterdag vanaf 10.00 uur tot 1...,Van maandag t/m zaterdag vanaf 10 uur tot 10 u...,KvK,KvK,Kamer van Koophandel,Netherlands,17173294
294,Bodybasics4kidz,KvK-nummer : 51629364. BTW-identificatienummer...,KvK-nummer : 51629364. BTW-identificatienummer...,KvK,KvK,Kamer van Koophandel,Netherlands,51629364


# VAT Number

### 1. Load Knowledge Base

In [12]:
# Load VAT Rules
vat_rules = pd.read_csv('VAT_RULES.csv', delimiter=';', encoding='UTF-8-SIG', dtype={'char':int})
# vat_rules = pd.read_csv('VAT_RULES_EUROPE.csv', delimiter=';', encoding='UTF-8-SIG', dtype={'char':int})
vat_rules.sample(3)

,group,country,country_code,code_format,char
93,Latin American Countries,Ecuador,EC,EC,13
28,European Union,Poland,PL,PL,13
65,Non-EU Countries,San Marino,SM,SM,5


### 2. Preprocess Knowledge Base

In [13]:
# Regex to filter VAT
def create_regex(row):
    code_format = row['code_format']
    char_digit = row['char']
    # regex = rf'\b{code_format}\s?[A-Z0-9]{{{char_digit}}}\b'
    regex = rf'\b{code_format}(?:\s?[A-Z\d]{{1,{char_digit}}})+\b'
    
    return re.compile(regex)

vat_rules['VAT_REGEX'] = vat_rules.apply(create_regex,axis=1)
vat_rules.sample(3)

,group,country,country_code,code_format,char,VAT_REGEX
98,Latin American Countries,Mexico,MX,MX,12,"re.compile('\\bMX(?:\\s?[A-Z\\d]{1,12})+\\b')"
90,Latin American Countries,Costa Rica,CR,CR,12,"re.compile('\\bCR(?:\\s?[A-Z\\d]{1,12})+\\b')"
55,Non-EU Countries,Japan,JP,JP,13,"re.compile('\\bJP(?:\\s?[A-Z\\d]{1,13})+\\b')"


### 3. Find VAT

In [14]:
# Create Function to filter VAT
patterns = vat_rules['VAT_REGEX'].to_list()
def find_vat_number(text):
    results = []
    if type(text) == str:
        text = text.replace('. ',' . ')

        for pattern in patterns:
            matches = pattern.findall(text)
            results.extend(matches)
    
    # remove duplicate
    clean_results = [res.strip() for res in results]
    clean_results = list(set(clean_results))

    # remove word
    pattern = r'\b(?:[A-Z]{4,})\b'
    clean_results = [re.sub(pattern, '', res).strip() for res in clean_results]
    
    # remove below 3 digits
    clean_results = [res for res in clean_results if len(res) > 3]
    
    return clean_results

comp_snippet_va = comp_snippet.copy()
comp_snippet_va['VAT_NUMBER'] = comp_snippet_va['CLEAN_SNIPPET'].apply(find_vat_number)

In [15]:
comp_snippet_va_expl = comp_snippet_va.explode('VAT_NUMBER')
comp_snippet_va_expl.sample(2)

,Company,FULL_SNIPPET,CLEAN_SNIPPET,BUSINESS_REGISTER,VAT_NUMBER
762,Muntex,Muntex BV Ootmarsumsestraat 15 7572 AA Oldenza...,Muntex BV Ootmarsumsestraat 15 7572 AA Oldenza...,[KvK],NL002393350B34
371,Michael Kors - EU,nan nan nan De prijzen van de Producten zijn z...,nan nan nan De prijzen van de Producten zijn z...,None,NaN


### 4. Find VAT Country

In [16]:
def find_vat_country(text):
    if type(text) == str:
        pattern = r'^[^\s^\d]+'

        match = re.search(pattern, text)
        if match:
            result = match.group(0)
            return result

comp_snippet_va_expl['code_format'] = comp_snippet_va_expl['VAT_NUMBER'].apply(find_vat_country)
comp_snippet_va_expl.sample(2)

,Company,FULL_SNIPPET,CLEAN_SNIPPET,BUSINESS_REGISTER,VAT_NUMBER,code_format
133,dress-for-less.nl,nan nan Inschrijving Kamer van Koophandel Darm...,nan nan Inschrijving Kamer van Koophandel Darm...,"[HRB, KvK]",DE311775086,DE
875,Foodworld-XL,Kamer van Koophandel: 09173779. BTW # NL: NL81...,Kamer van Koophandel: 09173779. BTW # NL: NL81...,[KvK],NL818362297B01 BTW,NL


In [17]:
comp_snippet_va_expl = comp_snippet_va_expl.merge(vat_rules[['country','code_format']],left_on='code_format',right_on='code_format',how='left')
comp_snippet_va_expl.sample(2)

,Company,FULL_SNIPPET,CLEAN_SNIPPET,BUSINESS_REGISTER,VAT_NUMBER,code_format,country
385,sokken.nl,fashcom B.V.. Louis Braillestraat 6-01 7442 DG...,fashcom .. Louis Braillestraat 6-01 7442 DG Ni...,[KvK],NL 85084901,NL,Netherlands
589,MyTrendyLady,nan nan nan Ons BTW-nummer is BG 205034841 (ht...,nan nan nan Ons BTW-nummer is BG 205034841 (ht...,None,BG 205034841,BG,Bulgaria


# Merge Business Register and VAT dataset

### 1. Merging Dataset

In [20]:
br_df = comp_snippet_br_expl[['country','Company','BUSINESS_REGISTER','BUSINESS_REGISTER_NUM']]

In [21]:
va_df = comp_snippet_va_expl[['country','Company','code_format','VAT_NUMBER']]

In [22]:
company_detail = pd.merge(br_df,va_df,how='outer',on=['country','Company'])
company_detail.drop_duplicates(inplace=True)
company_detail

,country,Company,BUSINESS_REGISTER,BUSINESS_REGISTER_NUM,code_format,VAT_NUMBER
0,Netherlands,De Bijenkorf,KvK,33116577,NL,NL003177397B01
1,Germany,Zalando.nl,HRB,158855 B,NaN,NaN
2,Germany,Zalando.nl,HRB,158855B,NaN,NaN
3,Netherlands,Zalando.nl,KvK,HRB 158855 B,NaN,NaN
4,Netherlands,Sliponline,KvK,KVK,NL,NL23RABO012842
...,...,...,...,...,...,...
2400,Netherlands,subtel.nl,NaN,NaN,NL,NL820698659B01
2401,NaN,eibabo.nl,NaN,NaN,HRB,HRB 25845
2402,NaN,Airfryertotaal,NaN,NaN,None,NaN
2403,Denmark,Coolshop.nl,NaN,NaN,DK,DK26457602 VAT


### 2. Find VAT Number Name

In [23]:
company_detail = company_detail.merge(knowledge[['country','vat_abb']],left_on='country',right_on='country')
company_detail = company_detail[['country','Company','BUSINESS_REGISTER','BUSINESS_REGISTER_NUM','vat_abb','VAT_NUMBER']]

In [32]:
company_detail.drop_duplicates(inplace=True)
company_detail.sample(3)

,Country,Company,Business register name,Business number,VAT number name,VAT number
437,Netherlands,Azerty,KvK,55425437,BTW,NL04RABO0120615797 BTW
259,Netherlands,Schoenenoutletonline.nl,KvK,16017276,BTW,NL009684967B01
1108,Germany,Van Dijk Waalwijk,HRB,423,USt-ID,NaN


### 3. Change Column Name

In [30]:
company_detail.rename(columns={
    'country':'Country',
    'BUSINESS_REGISTER':'Business register name',
    'BUSINESS_REGISTER_NUM':'Business number',
    'vat_abb':'VAT number name',
    'VAT_NUMBER':'VAT number'},inplace=True)
company_detail

,Country,Company,Business register name,Business number,VAT number name,VAT number
0,Netherlands,De Bijenkorf,KvK,33116577,BTW,NL003177397B01
1,Netherlands,Zalando.nl,KvK,HRB 158855 B,BTW,NaN
2,Netherlands,Sliponline,KvK,KVK,BTW,NL23RABO012842
3,Netherlands,Sliponline,KvK,17044923,BTW,NL23RABO012842
4,Netherlands,Naron,KvK,20081747,BTW,NL804811179B01
...,...,...,...,...,...,...
1850,Bulgaria,MyTrendyLady,NaN,NaN,ДДС,BG 205034841
1851,Bulgaria,MyTrendyLady,NaN,NaN,DDS,BG 205034841
1852,Hungary,dakraam-gordijn.nl,NaN,NaN,ÁFA,HU13543970
1853,Italy,Arredatutto.com,NaN,NaN,IVA,IT05835900969


# Save to CSV

In [31]:
company_detail.to_csv('company_detail.csv',index=False)